In [1]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from google.colab import drive
#drive.mount('/content/gdrive')

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 약간의 전처리 과정

In [115]:
df = pd.read_csv('NA_채우기_전.csv')
#df['you_rank_g'] = df['you_rank_g'].astype(str)
#df['previous_ranking'] = df['previous_ranking'].astype(str)

In [116]:
pd.set_option('display.max_columns', 999)

In [117]:
df.head()

,artist,name,rank_g,previous_ranking,genre,season,sex,song_type,active_type,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,title_song,week,you_rank_g
0,(여자)아이들,LATATA,2,2.0,댄스,summer,female,K_POP,Group,203,24,20.0,9.2,NaN,-0.093812,2.3,NaN,NaN,NaN,NaN,True,2018.7.1,10
1,(여자)아이들,LATATA,2,2.0,댄스,summer,female,K_POP,Group,203,24,20.0,12.0,NaN,-0.093812,2.4,NaN,NaN,NaN,NaN,True,2018.6.24,10
2,(여자)아이들,LATATA,2,4.0,댄스,spring,female,K_POP,Group,203,24,59.0,34.9,NaN,0.012954,3.9,NaN,NaN,NaN,NaN,True,2018.5.13,10
3,(여자)아이들,LATATA,7,6.0,댄스,fall,female,K_POP,Group,203,24,58.0,11.1,4310000.0,-0.020121,4.6,NaN,NaN,NaN,NaN,False,2018.9.16,4
4,(여자)아이들,LATATA,4,4.0,댄스,summer,female,K_POP,Group,203,24,80.0,100.0,8730000.0,-0.093812,9.1,NaN,NaN,NaN,NaN,False,2018.8.12,2


In [119]:
# null값 제거
df.loc[df['total_view'].isnull(),'total_view'] = 0
df.loc[df['dc_total_numb'].isnull(),'dc_total_numb'] = 0
df.loc[df['dc_mean_views'].isnull(),'dc_mean_views'] = 0
df.loc[df['dc_mean_reccomend'].isnull(),'dc_mean_reccomend'] = 0
df.loc[df['drama_view'].isnull(),'drama_view'] = 0
df.loc[df['previous_ranking'].isnull(),'previous_ranking'] = '10'

In [ ]:
# string으로 바꿔주기
df['rank_g'] = df['rank_g'].apply(lambda x: str(int(x)))
df['previous_ranking'] = df['previous_ranking'].apply(lambda x: str(int(x)))
df['you_rank_g'] = df['you_rank_g'].apply(lambda x: str(int(x)))

In [95]:
df = df.drop(columns = ['artist','name'])

In [121]:
# 장르 전부다 영어로
df.loc[df['genre'] == '댄스', 'genre'] = 'Dance'
df.loc[df['genre'] == '전체', 'genre'] = 'All'
df.loc[df['genre'] == '랩/힙합', 'genre'] = 'Rap/Hiphop'
df.loc[df['genre'] == '드라마', 'genre'] = 'Drama'
df.loc[df['genre'] == '인디', 'genre'] = 'Indi'
df.loc[df['genre'] == '팝', 'genre'] = 'Pop'
df.loc[df['genre'] == 'R&B/소울', 'genre'] = 'R&B/Soul'
df.loc[df['genre'] == '일렉트로니카', 'genre'] = 'Electronica'
df.loc[df['genre'] == '캐롤', 'genre'] = 'carol'
df.loc[df['genre'] == '락', 'genre'] = 'Rock'
df.loc[df['genre'] == '애니메이션/게임', 'genre'] = 'Ani/Game'
df.loc[df['genre'] == '발라드', 'genre'] = 'Ballad'
df.loc[df['genre'] == '해외영화', 'genre'] = 'Foreign Movie'
df.loc[df['genre'] == '트로트', 'genre'] = 'Trot'
df.loc[df['genre'] == '블루스/포크', 'genre'] = 'Blues/Folk'
df.loc[df['genre'] == '정통', 'genre'] = 'Jazz'

In [96]:
# 2020.4.19를 테스트로 한다. 
test = df.loc[df['week'] == '2020.4.19']
train = df.loc[df['week'] != '2020.4.19']

In [97]:
# week 필요없음
train = train.drop(columns = ['week'])
test = test.drop(columns = ['week'])

In [7]:
from sklearn.model_selection import train_test_split

In [98]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns = ['rank_g']),train['rank_g'], test_size = 0.3)

# 사용 예정 모델
- XGboost
- LightGBM
- RandomForest
- Catboost
- Logistic regression

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Logistic Regression

In [0]:
model = LogisticRegression(random_state=1)

In [0]:
stf = StratifiedKFold(n_splits = 2, shuffle = True, random_state = 1)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')
score.mean()

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-pa

0.6997561050303264

In [0]:
model.fit(X_train,y_train)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_y = model.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.689


# RandomForest

In [0]:
model = RandomForestClassifier(random_state =1, n_estimators = 500)

In [0]:
stf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [0]:
score.mean()

0.6888141663812283

In [0]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [0]:
pred_y = model.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.679


# Catboost

In [60]:
#!pip install catboost
from catboost import CatBoostClassifier

In [100]:
cat = [0,1,2,3,4,5,18]

In [101]:
X_train.head()

,previous_ranking,genre,season,sex,song_type,active_type,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,title_song,you_rank_g
5573,2,Drama,fall,female,OST,Solo,229,38,54.0,1.2,0.0,-0.107449,11.0,6.0,0.000000,257.666667,0.0,True,10
15060,3,Dance,spring,female,K_POP,Solo,213,62,31.0,4.8,8060000.0,0.012954,41.7,2593.0,0.730211,65.568587,0.0,True,2
7797,10,Indi,winter,male,K_POP,Duet,261,101,9.0,0.5,0.0,0.064639,12.5,0.0,0.000000,0.000000,0.0,True,10
3858,10,Ani/Game,winter,male,OST,Project,158,38,57.0,60.8,0.0,-0.122606,0.0,0.0,0.000000,0.000000,0.0,True,10
8115,10,Ballad,summer,male,K_POP,Group,232,63,46.0,2.5,0.0,-0.038576,3.3,50.0,1.976744,261.465116,0.0,True,10


In [102]:
model = CatBoostClassifier(n_estimators= 5000, learning_rate = 0.001, border_count = 7, depth = 10, random_state= 1,cat_features = cat)

In [103]:
model.fit(X_train,y_train)

0:	learn: 2.2980709	total: 437ms	remaining: 36m 24s
1:	learn: 2.2935936	total: 662ms	remaining: 27m 35s
2:	learn: 2.2890439	total: 1.26s	remaining: 35m 3s
3:	learn: 2.2846842	total: 1.75s	remaining: 36m 23s
4:	learn: 2.2803393	total: 2.07s	remaining: 34m 31s
5:	learn: 2.2759478	total: 2.54s	remaining: 35m 19s
6:	learn: 2.2716330	total: 2.88s	remaining: 34m 11s
7:	learn: 2.2672843	total: 3.34s	remaining: 34m 43s
8:	learn: 2.2628888	total: 3.87s	remaining: 35m 46s
9:	learn: 2.2587108	total: 4.24s	remaining: 35m 16s
10:	learn: 2.2546449	total: 4.64s	remaining: 35m 7s
11:	learn: 2.2507849	total: 5.04s	remaining: 34m 54s
12:	learn: 2.2465386	total: 5.43s	remaining: 34m 43s
13:	learn: 2.2427208	total: 5.52s	remaining: 32m 46s
14:	learn: 2.2386455	total: 5.85s	remaining: 32m 25s
15:	learn: 2.2347079	total: 6.21s	remaining: 32m 12s
16:	learn: 2.2308121	total: 6.56s	remaining: 32m 2s
17:	learn: 2.2268506	total: 6.92s	remaining: 31m 53s
18:	learn: 2.2230218	total: 7.09s	remaining: 30m 58s
19:	le

156:	learn: 1.8437794	total: 52.5s	remaining: 27m
157:	learn: 1.8417978	total: 52.9s	remaining: 27m
158:	learn: 1.8397531	total: 53.2s	remaining: 27m
159:	learn: 1.8377762	total: 53.4s	remaining: 26m 54s
160:	learn: 1.8359194	total: 53.8s	remaining: 26m 55s
161:	learn: 1.8338684	total: 54.1s	remaining: 26m 55s
162:	learn: 1.8319042	total: 54.5s	remaining: 26m 55s
163:	learn: 1.8299621	total: 54.6s	remaining: 26m 48s
164:	learn: 1.8280002	total: 54.9s	remaining: 26m 49s
165:	learn: 1.8260617	total: 55.3s	remaining: 26m 50s
166:	learn: 1.8241519	total: 55.7s	remaining: 26m 50s
167:	learn: 1.8222294	total: 55.8s	remaining: 26m 44s
168:	learn: 1.8203082	total: 55.9s	remaining: 26m 37s
169:	learn: 1.8183382	total: 56.3s	remaining: 26m 38s
170:	learn: 1.8163582	total: 56.7s	remaining: 26m 41s
171:	learn: 1.8144739	total: 56.8s	remaining: 26m 35s
172:	learn: 1.8126142	total: 57.4s	remaining: 26m 41s
173:	learn: 1.8107474	total: 57.5s	remaining: 26m 34s
174:	learn: 1.8089288	total: 57.6s	remai

307:	learn: 1.6042874	total: 1m 44s	remaining: 26m 35s
308:	learn: 1.6030287	total: 1m 45s	remaining: 26m 36s
309:	learn: 1.6017537	total: 1m 45s	remaining: 26m 38s
310:	learn: 1.6003852	total: 1m 46s	remaining: 26m 40s
311:	learn: 1.5991734	total: 1m 46s	remaining: 26m 36s
312:	learn: 1.5978041	total: 1m 46s	remaining: 26m 38s
313:	learn: 1.5965709	total: 1m 46s	remaining: 26m 36s
314:	learn: 1.5953409	total: 1m 47s	remaining: 26m 34s
315:	learn: 1.5942479	total: 1m 47s	remaining: 26m 30s
316:	learn: 1.5931960	total: 1m 47s	remaining: 26m 25s
317:	learn: 1.5919166	total: 1m 47s	remaining: 26m 26s
318:	learn: 1.5906957	total: 1m 48s	remaining: 26m 27s
319:	learn: 1.5893493	total: 1m 48s	remaining: 26m 28s
320:	learn: 1.5881952	total: 1m 48s	remaining: 26m 24s
321:	learn: 1.5870274	total: 1m 48s	remaining: 26m 22s
322:	learn: 1.5858555	total: 1m 49s	remaining: 26m 18s
323:	learn: 1.5847110	total: 1m 49s	remaining: 26m 19s
324:	learn: 1.5834829	total: 1m 49s	remaining: 26m 19s
325:	learn

457:	learn: 1.4444343	total: 2m 39s	remaining: 26m 19s
458:	learn: 1.4435186	total: 2m 39s	remaining: 26m 20s
459:	learn: 1.4426454	total: 2m 40s	remaining: 26m 21s
460:	learn: 1.4417154	total: 2m 40s	remaining: 26m 21s
461:	learn: 1.4408785	total: 2m 41s	remaining: 26m 22s
462:	learn: 1.4400344	total: 2m 41s	remaining: 26m 23s
463:	learn: 1.4391463	total: 2m 41s	remaining: 26m 23s
464:	learn: 1.4382315	total: 2m 42s	remaining: 26m 24s
465:	learn: 1.4373400	total: 2m 42s	remaining: 26m 24s
466:	learn: 1.4364758	total: 2m 43s	remaining: 26m 24s
467:	learn: 1.4356418	total: 2m 43s	remaining: 26m 24s
468:	learn: 1.4347591	total: 2m 44s	remaining: 26m 25s
469:	learn: 1.4338278	total: 2m 44s	remaining: 26m 25s
470:	learn: 1.4328917	total: 2m 44s	remaining: 26m 25s
471:	learn: 1.4320180	total: 2m 45s	remaining: 26m 26s
472:	learn: 1.4311719	total: 2m 45s	remaining: 26m 26s
473:	learn: 1.4302705	total: 2m 46s	remaining: 26m 26s
474:	learn: 1.4293813	total: 2m 46s	remaining: 26m 26s
475:	learn

609:	learn: 1.3267864	total: 3m 35s	remaining: 25m 53s
610:	learn: 1.3260785	total: 3m 36s	remaining: 25m 52s
611:	learn: 1.3254349	total: 3m 36s	remaining: 25m 51s
612:	learn: 1.3247308	total: 3m 36s	remaining: 25m 51s
613:	learn: 1.3240244	total: 3m 37s	remaining: 25m 50s
614:	learn: 1.3233799	total: 3m 37s	remaining: 25m 49s
615:	learn: 1.3227587	total: 3m 37s	remaining: 25m 49s
616:	learn: 1.3221639	total: 3m 37s	remaining: 25m 47s
617:	learn: 1.3215275	total: 3m 38s	remaining: 25m 46s
618:	learn: 1.3208047	total: 3m 38s	remaining: 25m 46s
619:	learn: 1.3200930	total: 3m 38s	remaining: 25m 46s
620:	learn: 1.3193706	total: 3m 39s	remaining: 25m 46s
621:	learn: 1.3187492	total: 3m 39s	remaining: 25m 47s
622:	learn: 1.3181288	total: 3m 39s	remaining: 25m 44s
623:	learn: 1.3174525	total: 3m 40s	remaining: 25m 45s
624:	learn: 1.3167849	total: 3m 40s	remaining: 25m 45s
625:	learn: 1.3161860	total: 3m 41s	remaining: 25m 46s
626:	learn: 1.3155227	total: 3m 41s	remaining: 25m 46s
627:	learn

...

4274:	learn: 0.7865657	total: 42m 25s	remaining: 7m 11s
4275:	learn: 0.7865311	total: 42m 26s	remaining: 7m 11s
4276:	learn: 0.7865078	total: 42m 27s	remaining: 7m 10s
4277:	learn: 0.7864769	total: 42m 29s	remaining: 7m 10s
4278:	learn: 0.7864422	total: 42m 30s	remaining: 7m 9s
4279:	learn: 0.7864136	total: 42m 31s	remaining: 7m 9s
4280:	learn: 0.7863778	total: 42m 32s	remaining: 7m 8s
4281:	learn: 0.7863226	total: 42m 33s	remaining: 7m 8s
4282:	learn: 0.7862906	total: 42m 34s	remaining: 7m 7s
4283:	learn: 0.7862555	total: 42m 34s	remaining: 7m 7s
4284:	learn: 0.7862110	total: 42m 35s	remaining: 7m 6s
4285:	learn: 0.7861809	total: 42m 36s	remaining: 7m 5s
4286:	learn: 0.7861409	total: 42m 37s	remaining: 7m 5s
4287:	learn: 0.7860911	total: 42m 38s	remaining: 7m 4s
4288:	learn: 0.7860614	total: 42m 39s	remaining: 7m 4s
4289:	learn: 0.7860137	total: 42m 39s	remaining: 7m 3s
4290:	learn: 0.7859719	total: 42m 40s	remaining: 7m 3s
4291:	learn: 0.7859303	total: 42m 41s	remaining: 7m 2s
4292:	

4422:	learn: 0.7813377	total: 44m 51s	remaining: 5m 51s
4423:	learn: 0.7812888	total: 44m 51s	remaining: 5m 50s
4424:	learn: 0.7812755	total: 44m 52s	remaining: 5m 49s
4425:	learn: 0.7812438	total: 44m 53s	remaining: 5m 49s
4426:	learn: 0.7811850	total: 44m 54s	remaining: 5m 48s
4427:	learn: 0.7811457	total: 44m 55s	remaining: 5m 48s
4428:	learn: 0.7811183	total: 44m 56s	remaining: 5m 47s
4429:	learn: 0.7810932	total: 44m 57s	remaining: 5m 47s
4430:	learn: 0.7810706	total: 44m 58s	remaining: 5m 46s
4431:	learn: 0.7810407	total: 44m 59s	remaining: 5m 45s
4432:	learn: 0.7810089	total: 45m	remaining: 5m 45s
4433:	learn: 0.7809635	total: 45m 1s	remaining: 5m 44s
4434:	learn: 0.7809300	total: 45m 2s	remaining: 5m 44s
4435:	learn: 0.7809049	total: 45m 3s	remaining: 5m 43s
4436:	learn: 0.7808698	total: 45m 4s	remaining: 5m 43s
4437:	learn: 0.7808373	total: 45m 6s	remaining: 5m 42s
4438:	learn: 0.7808131	total: 45m 7s	remaining: 5m 42s
4439:	learn: 0.7807710	total: 45m 7s	remaining: 5m 41s
444

4570:	learn: 0.7762583	total: 46m 59s	remaining: 4m 24s
4571:	learn: 0.7762123	total: 47m	remaining: 4m 24s
4572:	learn: 0.7761911	total: 47m 1s	remaining: 4m 23s
4573:	learn: 0.7761653	total: 47m 1s	remaining: 4m 22s
4574:	learn: 0.7761317	total: 47m 2s	remaining: 4m 22s
4575:	learn: 0.7760998	total: 47m 3s	remaining: 4m 21s
4576:	learn: 0.7760682	total: 47m 4s	remaining: 4m 21s
4577:	learn: 0.7760270	total: 47m 5s	remaining: 4m 20s
4578:	learn: 0.7759807	total: 47m 5s	remaining: 4m 19s
4579:	learn: 0.7759682	total: 47m 6s	remaining: 4m 19s
4580:	learn: 0.7759270	total: 47m 7s	remaining: 4m 18s
4581:	learn: 0.7758715	total: 47m 8s	remaining: 4m 17s
4582:	learn: 0.7758159	total: 47m 8s	remaining: 4m 17s
4583:	learn: 0.7757722	total: 47m 9s	remaining: 4m 16s
4584:	learn: 0.7757215	total: 47m 10s	remaining: 4m 16s
4585:	learn: 0.7756833	total: 47m 11s	remaining: 4m 15s
4586:	learn: 0.7756472	total: 47m 11s	remaining: 4m 14s
4587:	learn: 0.7756054	total: 47m 12s	remaining: 4m 14s
4588:	le

4718:	learn: 0.7711847	total: 49m 6s	remaining: 2m 55s
4719:	learn: 0.7711626	total: 49m 7s	remaining: 2m 54s
4720:	learn: 0.7711374	total: 49m 7s	remaining: 2m 54s
4721:	learn: 0.7711093	total: 49m 8s	remaining: 2m 53s
4722:	learn: 0.7710802	total: 49m 10s	remaining: 2m 53s
4723:	learn: 0.7710651	total: 49m 10s	remaining: 2m 52s
4724:	learn: 0.7710321	total: 49m 11s	remaining: 2m 51s
4725:	learn: 0.7710036	total: 49m 12s	remaining: 2m 51s
4726:	learn: 0.7709740	total: 49m 13s	remaining: 2m 50s
4727:	learn: 0.7709457	total: 49m 14s	remaining: 2m 49s
4728:	learn: 0.7709281	total: 49m 15s	remaining: 2m 49s
4729:	learn: 0.7708870	total: 49m 16s	remaining: 2m 48s
4730:	learn: 0.7708572	total: 49m 17s	remaining: 2m 48s
4731:	learn: 0.7708330	total: 49m 18s	remaining: 2m 47s
4732:	learn: 0.7708047	total: 49m 19s	remaining: 2m 46s
4733:	learn: 0.7707722	total: 49m 20s	remaining: 2m 46s
4734:	learn: 0.7707439	total: 49m 21s	remaining: 2m 45s
4735:	learn: 0.7707111	total: 49m 22s	remaining: 2m 

4866:	learn: 0.7664274	total: 51m 14s	remaining: 1m 24s
4867:	learn: 0.7663890	total: 51m 15s	remaining: 1m 23s
4868:	learn: 0.7663557	total: 51m 16s	remaining: 1m 22s
4869:	learn: 0.7663174	total: 51m 17s	remaining: 1m 22s
4870:	learn: 0.7662853	total: 51m 18s	remaining: 1m 21s
4871:	learn: 0.7662580	total: 51m 19s	remaining: 1m 20s
4872:	learn: 0.7662287	total: 51m 19s	remaining: 1m 20s
4873:	learn: 0.7661785	total: 51m 20s	remaining: 1m 19s
4874:	learn: 0.7661535	total: 51m 22s	remaining: 1m 19s
4875:	learn: 0.7661286	total: 51m 23s	remaining: 1m 18s
4876:	learn: 0.7661179	total: 51m 23s	remaining: 1m 17s
4877:	learn: 0.7660878	total: 51m 23s	remaining: 1m 17s
4878:	learn: 0.7660387	total: 51m 24s	remaining: 1m 16s
4879:	learn: 0.7659948	total: 51m 25s	remaining: 1m 15s
4880:	learn: 0.7659845	total: 51m 26s	remaining: 1m 15s
4881:	learn: 0.7659500	total: 51m 26s	remaining: 1m 14s
4882:	learn: 0.7659265	total: 51m 27s	remaining: 1m 13s
4883:	learn: 0.7658794	total: 51m 28s	remaining:

In [104]:
pred_y = model.predict(X_test)

In [113]:
# 최종 결과
print("F1 : %.3f" % f1_score(test['rank_g_pred'], test['rank_g'], average = 'micro'))

F1 : 0.760


In [107]:
import joblib

In [ ]:
pred_y = model.predict(X_test)

In [125]:
df['rank_g_pred'] = model.predict(df.drop(columns = ['rank_g','artist','name','week']))

In [126]:
df.head()

,artist,name,rank_g,previous_ranking,genre,season,sex,song_type,active_type,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view,title_song,week,you_rank_g,rank_g_pred
0,(여자)아이들,LATATA,2,2,Dance,summer,female,K_POP,Group,203,24,20.0,9.2,0.0,-0.093812,2.3,0.0,0.0,0.0,0.0,True,2018.7.1,10,2
1,(여자)아이들,LATATA,2,2,Dance,summer,female,K_POP,Group,203,24,20.0,12.0,0.0,-0.093812,2.4,0.0,0.0,0.0,0.0,True,2018.6.24,10,2
2,(여자)아이들,LATATA,2,4,Dance,spring,female,K_POP,Group,203,24,59.0,34.9,0.0,0.012954,3.9,0.0,0.0,0.0,0.0,True,2018.5.13,10,4
3,(여자)아이들,LATATA,7,6,Dance,fall,female,K_POP,Group,203,24,58.0,11.1,4310000.0,-0.020121,4.6,0.0,0.0,0.0,0.0,False,2018.9.16,4,7
4,(여자)아이들,LATATA,4,4,Dance,summer,female,K_POP,Group,203,24,80.0,100.0,8730000.0,-0.093812,9.1,0.0,0.0,0.0,0.0,False,2018.8.12,2,4


In [128]:
# 최종 결과 저장
df.to_csv('최종 결과.csv')

In [127]:
# 
print("F1 : %.3f" % f1_score(df['rank_g_pred'], df['rank_g'], average = 'micro'))

F1 : 0.725


In [108]:
joblib.dump(model,'cpu_version1.pkl')

['cpu_version1.pkl']

# Xgboost

In [0]:
from xgboost import XGBClassifier

In [0]:
# XGB는 생각보다 성능이 낮다... 아마 파라미터 조정을 해야될듯...
model = XGBClassifier(n_estimators = 500,learning_rate = 0.05)

In [0]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [0]:
score.mean()

0.6954003804672746

In [0]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [0]:
pred_y = model.predict(X_test)
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.687


In [0]:
import joblib

In [0]:
clf_from_joblib = joblib.load('final_model.pkl') 

In [0]:
pred_y = clf_from_joblib.predict(X_test)

In [0]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.719
